In [30]:
import requests
from typing import Dict, Any

In [ ]:
from typing import Dict, Any
import requests


def get_weather_forecast(location: str, days: int = 3) -> Dict[str, Any]:
    geo = requests.get(
        "https://geocoding-api.open-meteo.com/v1/search",
        params={"name": location, "count": 1},
        timeout=10,
    ).json()

    if not geo.get("results"):
        raise ValueError(f"Location not found: {location}")

    loc = geo["results"][0]
    lat, lon = loc["latitude"], loc["longitude"]

    weather = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={
            "latitude": lat,
            "longitude": lon,
            "hourly": [
                "temperature_2m",
                "relative_humidity_2m",
                "wind_speed_10m",
                "shortwave_radiation",
                "cloud_cover",
            ],
            "current": [
                "temperature_2m",
                "relative_humidity_2m",
                "wind_speed_10m",
                "cloud_cover",
            ],
            "forecast_days": days,
            "timezone": "auto",
        },
        timeout=10,
    ).json()

    # Building response
    hourly = weather["hourly"]
    hours = min(days * 24, len(hourly["temperature_2m"]))

    forecast = {
        "location": f'{loc["name"]}, {loc["country"]}',
        "forecast_days": days,
        "current": {
            "temperature_c": weather["current"]["temperature_2m"],
            "condition": "cloudy" if weather["current"]["cloud_cover"] > 50 else "sunny",
            "cloud_cover": weather["current"]["cloud_cover"],
            #"solar_irradiance": weather["current"]["shortwave_radiation"],
            "humidity": weather["current"]["relative_humidity_2m"],
            "wind_speed": weather["current"]["wind_speed_10m"],
        },
        "hourly_units": weather["hourly_units"],
        "hourly": [],
    }

    for i in range(hours):
        forecast["hourly"].append(
            {
                "time": hourly["time"][i],
                "temperature_c": hourly["temperature_2m"][i],
                "condition": "cloudy" if hourly["cloud_cover"][i] > 50 else "sunny",
                "cloud_cover": hourly["cloud_cover"][i],
                "solar_irradiance": hourly["shortwave_radiation"][i],
                "humidity": hourly["relative_humidity_2m"][i],
                "wind_speed": hourly["wind_speed_10m"][i],
            }
        )

    return forecast


In [32]:
from rich.pretty import Pretty
Pretty(get_weather_forecast("tokyo"))

NameError: name 'loc' is not defined